In [ ]:
from faker import Faker
import mysql.connector
import re
import unicodedata
from datetime import date, timedelta
import random

con = mysql.connector.connect(
    host="giniewicz.it",
    user="team19",
    password="te@mzaiq",
    database="team19"
)

mycursor = con.cursor()

# Usunięcie danych z każdej tabeli
tabele = ["Klienci", "Pracownicy", "Transakcje_finansowe", "Koszty_Organizacji"]
for tabela in tabele:
    mycursor.execute(f"DELETE FROM {tabela}")
    mycursor.execute(f"ALTER TABLE {tabela} AUTO_INCREMENT = 1")  # Resetowanie AUTO_INCREMENT

con.commit()  # Zatwierdzenie usunięcia danych

fake = Faker("pl_PL")

def usun_polskie_znaki(tekst):
    polskie_znaki = {
        'ą': 'a', 'ć': 'c', 'ę': 'e', 'ł': 'l', 'ń': 'n', 'ó': 'o', 'ś': 's', 'ź': 'z', 'ż': 'z',
        'Ą': 'A', 'Ć': 'C', 'Ę': 'E', 'Ł': 'L', 'Ń': 'N', 'Ó': 'O', 'Ś': 'S', 'Ź': 'Z', 'Ż': 'Z'
    }
    return ''.join(polskie_znaki.get(c, c) for c in tekst)

domeny = ["gmail.com", "yahoo.com", "outlook.com", "wp.pl", "interia.pl", "onet.pl"]

def generuj_email(imie, nazwisko):
    domena = random.choice(domeny)
    return f"{usun_polskie_znaki(imie.lower())}.{usun_polskie_znaki(nazwisko.lower())}@{domena}"

def generuj_pensje(stanowisko):
    if stanowisko == "Manager":
        return random.randint(12000, 15000)
    elif stanowisko in ["Programista", "Księgowy"]:
        return random.randint(10000, 12000)
    elif stanowisko == "Specjalista ds. marketingu":
        return random.randint(8000, 10000)
    else:
        return random.randint(5000, 8000)

def generuj_date_urodzenia(stanowisko):
    if stanowisko == "Manager":
        return fake.date_of_birth(minimum_age=30, maximum_age=45)
    elif stanowisko in ["Programista", "Księgowy", "Specjalista ds. marketingu"]:
        return fake.date_of_birth(minimum_age=25, maximum_age=45)
    else:
        return fake.date_of_birth(minimum_age=20, maximum_age=45)

def losuj_date_zatrudnienia(stanowisko):
    if stanowisko in ["Manager", "Programista"]:
        start_date = date(2022, 1, 1)
        end_date = date(2022, 1, 5)
    else:
        start_date = date(2022, 1, 5)
        end_date = date(2022, 1, 14)
    return start_date + timedelta(days=random.randint(0, (end_date - start_date).days))

for _ in range(300):
    Imie = fake.first_name()
    Nazwisko = fake.last_name()
    Email = generuj_email(Imie, Nazwisko)
    Numer_telefonu = "+48 " + " ".join([str(int(re.sub(r'\D', '', fake.phone_number())))[-9:].rjust(9, '0')[i:i+3] for i in range(0, 9, 3)])
    Adres = fake.address().replace("\n", ", ")
    Imie_bliskiej_osoby = fake.first_name()
    if random.random() < 0.8:
        Nazwisko_bliskiej_osoby = Nazwisko
    else:
        Nazwisko_bliskiej_osoby = fake.last_name()
    Numer_telefonu_bliskiej_osoby = "+48 " + " ".join([str(int(re.sub(r'\D', '', fake.phone_number())))[-9:].rjust(9, '0')[i:i+3] for i in range(0, 9, 3)])
    Dane_kontaktowe_bliskich = f"{Imie_bliskiej_osoby} {Nazwisko_bliskiej_osoby}, {Numer_telefonu_bliskiej_osoby}"
    Data_urodzenia = fake.date_of_birth(minimum_age=18, maximum_age=80).strftime('%Y-%m-%d')
    mycursor.execute(""" 
        INSERT INTO Klienci (Imie, Nazwisko, Numer_telefonu, Email, Adres, Dane_kontaktowe_bliskich, Data_urodzenia)
        VALUES (%s, %s, %s, %s, %s, %s, %s)
    """, (Imie, Nazwisko, Numer_telefonu, Email, Adres, Dane_kontaktowe_bliskich, Data_urodzenia))

stanowiska = [
    "Manager",
    "Programista",
    "Księgowy",
    "Specjalista ds. marketingu",
    "Konsultant ds. obsługi klienta",
    "Konsultant ds. obsługi klienta",
    "Konsultant ds. obsługi klienta"
]

for stanowisko in stanowiska:
    imie = fake.first_name()
    nazwisko = fake.last_name()
    email = generuj_email(imie, nazwisko)
    numer_telefonu = "+48 " + " ".join([str(int(re.sub(r'\D', '', fake.phone_number())))[-9:].rjust(9, '0')[i:i+3] for i in range(0, 9, 3)])
    adres = fake.address().replace("\n", ", ")
    pensja = generuj_pensje(stanowisko)
    data_urodzenia = generuj_date_urodzenia(stanowisko)
    data_zatrudnienia = losuj_date_zatrudnienia(stanowisko)
    mycursor.execute(""" 
        INSERT INTO Pracownicy (Imie, Nazwisko, Stanowisko, Numer_telefonu, Email, Adres, Pensja, Data_urodzenia, Data_zatrudnienia)
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)
    """, (imie, nazwisko, stanowisko, numer_telefonu, email, adres, pensja, data_urodzenia, data_zatrudnienia))

insert_data_query = """
INSERT INTO Rodzaje_wycieczek (Nazwa, Opis) 
VALUES (%s, %s);
"""

wycieczki = [
    ("Wielka Rafa Koralowa i podwodne przygody", 
     "Odkryj podwodny świat jednego z największych cudów natury! Czeka Cię rejs na Wielką Rafę Koralową, gdzie będziesz nurkować lub snorkelować wśród kolorowych ryb. W programie znajdziesz także wieczorny rejs z kolacją pod gwiazdami oraz wycieczkę do tropikalnego lasu Daintree. Dla bardziej odważnych przewidziano opcję nurkowania w klatce z rekinami (za dodatkową opłatą)."),
    
    ("Pustynna Odyseja", 
     "Wybierz się w podróż przez australijskie pustkowia, zaczynając od Alice Springs – serca Outbacku. Odwiedzisz tajemnicze formacje skalne, takie jak Devils Marbles, i spróbujesz lokalnych atrakcji, takich jak rzucanie bumerangiem. Następnie czeka Cię eksploracja Uluru i okolicznych parków narodowych, w tym przejażdżka wielbłądami przez pustynne wydmy. Wieczory spędzisz w wygodnych namiotach lub w hotelach w buszu, gdzie spróbujesz słynnych australijskich lamingtonów."),
    
    ("Survival w australijskim buszu", 
     "Gotowy na wyzwanie? Survival w buszu to wyprawa tylko dla odważnych! W sercu pustynnej Australii, w okolicach Alice Springs, nauczysz się, jak przetrwać w trudnych warunkach: znajdziesz wodę, zbudujesz schronienie z gałęzi i liści oraz rozpalisz ogień bez zapałek. Profesjonalni przewodnicy pokażą Ci, jak unikać niebezpiecznych zwierząt i rozpoznawać jadalne rośliny. Na koniec wyprawy zmierzysz się z “testem przetrwania” – spędzeniem jednej nocy samemu w buszu. Nie martw się – instruktorzy będą czuwać!"),
    
    ("Wielkomiejska przygoda w Sydney", 
     "Tydzień w Sydney to połączenie wielkomiejskiego życia, kultury i relaksu. Odwiedzisz najsłynniejsze miejsca, takie jak Opera w Sydney, most Harbour Bridge, czy przepiękna plaża Bondi. Wieczorami zanurzysz się w nocnym życiu Sydney, odwiedzając lokalne bary, gdzie spróbujesz rzemieślniczych piw i prawdziwych australijskich koktajli."),
    
    ("Surfing i plażowanie", 
     "Marzysz o złotych plażach i falach idealnych do surfingu? Na Gold Coast nauczysz się surfować z pomocą profesjonalnych instruktorów. Czeka Cię także wyprawa na Fraser Island – największą na świecie piaszczystą wyspę. Dni wypełnione surfingiem i opalaniem zakończą się wieczornymi ogniskami na plaży z widokiem na rozgwieżdżone niebo."),
    
    ("Adelajda i Winne Doliny", 
     "Tygodniowa wyprawa do Adelajdy to doskonałe połączenie miejskich atrakcji i wycieczek za miasto. Zwiedzisz eleganckie centrum Adelajdy, odwiedzając galerie sztuki, muzea i słynny Central Market, gdzie spróbujesz lokalnych smakołyków. Następnie wyruszysz do Doliny Barossa, jednej z najsłynniejszych australijskich krain winiarskich. Czeka Cię degustacja wina w kilku winiarniach oraz poznanie procesu jego produkcji. Kolejne dni spędzisz na Kangaroo Island, gdzie spotkasz dzikie zwierzęta, w tym lwy morskie i oczywiście kangury, a także zachwycisz się spektakularnymi widokami skalnych formacji i pustych plaż."),
    
    ("Melbourne: Kultura, Kawa i Street Art", 
     "Melbourne to mekka dla miłośników kultury i dobrej kawy. W trakcie wycieczki przemierzysz najbardziej znane zakątki tego miasta. Spacer po ukrytych pasażach handlowych i ulicach ozdobionych kolorowym street artem pozwoli Ci odkryć artystyczną duszę Melbourne. Zajrzymy do kawiarni w Fitzroy, aby poznać lokalne tradycje picia kawy, a potem zwiedzimy Galerię Narodową Wiktorii oraz słynny stadion MCG. Nie zabraknie też wycieczki na pobliską Phillip Island, gdzie wieczorem zobaczysz paradę pingwinów wracających na brzeg. Na zakończenie spróbujesz lokalnych przysmaków w modnych restauracjach i odwiedzisz dzielnicę St Kilda."),
    
    ("Wombaty i inne cuda australijskiej przyrody", 
     "Rozpocznij swoją australijską przygodę w Melbourne, skąd wyruszysz na zwiedzanie najpiękniejszych parków narodowych południowo-wschodniej Australii. Podczas wyprawy odwiedzisz rezerwat Wilsons Promontory, gdzie spotkasz wombaty, kangury i emu w ich naturalnym środowisku. Na trasie znajdzie się również Park Narodowy Grampians – idealne miejsce na trekking i podziwianie wodospadów MacKenzie. W programie przewidziano warsztaty fotograficzne z lokalnym przewodnikiem."),
    
    ("Miasta i cuda natury", 
     "Kompleksowa wyprawa, która łączy największe miasta i naturalne atrakcje Australii. Zaczynasz w Sydney, gdzie zwiedzisz słynny Opera House i pospacerujesz po plaży Bondi. Następnie udasz się do Melbourne, aby poznać jego artystyczną duszę. W planie jest również przejazd Great Ocean Road, czyli jednej z najpiękniejszych tras widokowych na świecie, z przystankiem przy Dwunastu Apostołach. W programie także trekking w Blue Mountains oraz relaks na złotych plażach Gold Coast."),
    
    ("Przygoda w Brisbane", 
     "Brisbane, z jego subtropikalnym klimatem i tętniącym życiem centrum, oferuje idealne połączenie relaksu i aktywności. W tygodniowym programie znajdziesz zwiedzanie centrum miasta, rejs po rzece Brisbane i wjazd na Mount Coot-tha, gdzie zachwyci Cię widok na panoramę miasta. Po drodze odwiedzisz Lone Pine Koala Sanctuary, gdzie przytulisz koalę i nakarmisz kangury. Czas wolny spędzisz w South Bank, tętniącym życiem miejscu nad rzeką, pełnym restauracji, barów i sztucznej miejskiej plaży. Na zakończenie wyruszysz na całodniową wycieczkę na Gold Coast, gdzie plaże i surfing stanowią kwintesencję australijskiego życia.")
]

def generuj_atrakcje():
    return round(random.uniform(300, 1500), 2)

def generuj_ubezpieczenie():
    return round(random.uniform(50, 300), 2)

def generuj_transport():
    return round(random.uniform(100, 800), 2)

def generuj_nocleg():
    return round(random.uniform(100, 500), 2)

def generuj_przewodnik():
    return round(random.uniform(200, 800), 2)

def generuj_loty():
    return round(random.uniform(1000, 5000), 2)

for id_rodzaj in range(1, 11):
    koszty = {
        "Nocleg_za_noc": generuj_nocleg(),
        "Przewodnik": generuj_przewodnik(),
        "Loty": generuj_loty(),
        "Atrakcje": generuj_atrakcje(),
        "Ubezpieczenie": generuj_ubezpieczenie(),
        "Transport": generuj_transport()
    }

    mycursor.execute('''
        INSERT INTO Koszty_Organizacji (id_rodzaj, Nocleg_za_noc, Przewodnik, Loty, Atrakcje, Ubezpieczenie, Transport)
        VALUES (%s, %s, %s, %s, %s, %s, %s)
    ''', (
        id_rodzaj,
        koszty["Nocleg_za_noc"],
        koszty["Przewodnik"],
        koszty["Loty"],
        koszty["Atrakcje"],
        koszty["Ubezpieczenie"],
        koszty["Transport"]
    ))
def generuj_czy_zrealizowano():
    return 1 if random.random() < random.uniform(0.93, 0.98) else 0

def generuj_metode_platnosci():
    wybor = random.random()
    if wybor < random.uniform(0.55, 0.65):
        return random.choice(["Przelew", "Blik"])
    elif wybor < random.uniform(0.70, 0.90):
        return "Płatność kartą"
    else:
        return random.choice(["Apple Pay", "Google Pay"])

for _ in range(1000):
    zrealizowano = generuj_czy_zrealizowano()
    metoda_platnosci = generuj_metode_platnosci()
    mycursor.execute(""" 
        INSERT INTO Transakcje_finansowe (Zrealizowano, Metoda_platnosci)
        VALUES (%s, %s)
    """, (zrealizowano, metoda_platnosci))

con.commit()
mycursor.close()
con.close()
